In [6]:
# Install these once (uncomment if needed)
!pip install langchain openai psycopg2-binary sqlalchemy

import os
from sqlalchemy import create_engine, text
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install langchain openai langchain-community --quiet

In [27]:
import os
from sqlalchemy import create_engine, text
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [28]:
# 📌 Step 1: Install Dependencies
!pip install psycopg2-binary pandas
!pip install pg8000 cloud-sql-python-connector

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [29]:
# 📌 Step 2: Import Libraries
import psycopg2
import pandas as pd
import os

In [30]:
# 📌 Step 3: Set Up Database Connection (Replace with Your Details)
DATABASE_URL = "dbname='postgres' user='postgres' host='34.59.119.208' password='avantichhaya'"

def connect_db():
    try:
        conn = psycopg2.connect(DATABASE_URL)
        print("✅ Connected to PostgreSQL Cloud SQL")
        return conn
    except Exception as e:
        print("🚨 Connection Error:", e)
        return None

conn = connect_db()
cursor = conn.cursor()

✅ Connected to PostgreSQL Cloud SQL


In [18]:
# Reset the connection to recover from the failed transaction
conn.rollback()
print("🔄 Connection rollback complete. You're good to go.")


🔄 Connection rollback complete. You're good to go.


In [7]:
def fetch_unclassified_email():
    query = """
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
        LIMIT 1
    """
    cursor.execute(query)
    row = cursor.fetchone()
    return row

email = fetch_unclassified_email()

if email:
    email_id, subject, body = email
    print("📩 Subject:", subject)
    print("📝 Body:", body[:300])
else:
    print("✅ No unclassified emails found.")

📩 Subject: help needed
📝 Body: how does this work? Help me


In [8]:
insert_query = """
    INSERT INTO emails (sender_email, subject, body)
    VALUES (%s, %s, %s)
"""
test_email = (
    "test.user@example.com",
    "Request for leave approval",
    "Hi HR, I’d like to apply for a leave from March 29th to April 2nd. Please let me know the procedure."
)

cursor.execute(insert_query, test_email)
conn.commit()
print("✅ Test email inserted.")

✅ Test email inserted.


In [16]:
def fetch_unclassified_email():
    query = """
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
        LIMIT 1
    """
    cursor.execute(query)
    row = cursor.fetchone()
    return row

email = fetch_unclassified_email()

if email:
    email_id, subject, body = email
    print("📩 Subject:", subject)
    print("📝 Body:", body[:300])
else:
    print("✅ No unclassified emails found.")

📩 Subject: Leave Request
📝 Body: I need leave for 2 days.


In [4]:
!pip install transformers --quiet

In [5]:
!pip install torch --quiet

In [15]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [11]:
email_text = f"{subject}\n\n{body}"
categories = ["Leave Request", "Job Offer", "Onboarding", "Benefits Inquiry", "General HR Query"]

result = classifier(email_text, categories)

predicted_label = result["labels"][0]
confidence = result["scores"][0]

print("🧠 Predicted Category:", predicted_label)
print("📈 Confidence Score:", round(confidence, 2))

🧠 Predicted Category: Onboarding
📈 Confidence Score: 0.44


In [26]:
cursor.execute("""
    UPDATE emails
    SET classified_category = %s
    WHERE email_id = %s
""", (predicted_label, email_id))
conn.commit()

print(f"✅ Email {email_id} updated as '{predicted_label}'")

✅ Email 8 updated as 'Leave Request'


In [27]:
def fetch_placeholder_emails(limit=5):
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category = 'aa'
        LIMIT %s
    """, (limit,))
    return cursor.fetchall()

emails_to_classify = fetch_placeholder_emails()

print(f"🔍 Found {len(emails_to_classify)} emails needing classification.")

🔍 Found 3 emails needing classification.


In [28]:
for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"

    result = classifier(email_text, categories)
    predicted_label = result["labels"][0]
    confidence = result["scores"][0]

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}\n")

    # Update the classified category
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))

conn.commit()
print("✅ All emails classified and updated.")

📩 Email ID 4
🧠 Predicted: Leave Request
📈 Confidence: 0.87

📩 Email ID 5
🧠 Predicted: Onboarding
📈 Confidence: 0.32

📩 Email ID 6
🧠 Predicted: Onboarding
📈 Confidence: 0.64

✅ All emails classified and updated.


**NEW MODEL**

In [6]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"
)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cpu


In [14]:
cursor.execute("""
    UPDATE emails
    SET classified_category = NULL
""")
conn.commit()

print("✅ All classified categories reset to NULL.")

✅ All classified categories reset to NULL.


In [18]:
categories = [
    "Leave Request",
    "Payroll Inquiry",
    "Onboarding",
    "Technical Issue",   # Optional, based on email content
    "Help Request"
]
def fetch_unclassified_emails():
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """)
    return cursor.fetchall()

emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"
    result = classifier(email_text, categories)
    predicted_label = result["labels"][0]
    confidence = result["scores"][0]

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}")

    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))

conn.commit()
print("✅ Reclassification complete.")

📩 Email ID 1
🧠 Predicted: Leave Request
📈 Confidence: 0.99
📩 Email ID 2
🧠 Predicted: Technical Issue
📈 Confidence: 0.95
📩 Email ID 3
🧠 Predicted: Leave Request
📈 Confidence: 0.39
📩 Email ID 7
🧠 Predicted: Leave Request
📈 Confidence: 0.93
📩 Email ID 10
🧠 Predicted: Help Request
📈 Confidence: 0.52
📩 Email ID 11
🧠 Predicted: Leave Request
📈 Confidence: 0.93
📩 Email ID 8
🧠 Predicted: Leave Request
📈 Confidence: 0.93
📩 Email ID 4
🧠 Predicted: Leave Request
📈 Confidence: 0.99
📩 Email ID 5
🧠 Predicted: Technical Issue
📈 Confidence: 0.95
📩 Email ID 6
🧠 Predicted: Leave Request
📈 Confidence: 0.39
📩 Email ID 12
🧠 Predicted: Leave Request
📈 Confidence: 0.93
✅ Reclassification complete.


**Resetting the status to null**

In [20]:
cursor.execute("""
    UPDATE emails
    SET classified_category = NULL
""")
conn.commit()

print("✅ All classified categories reset to NULL.")

✅ All classified categories reset to NULL.


**Classfication with new Categories**

In [21]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [22]:
categories = [
    "Leave Request",
    "Onboarding",
    "Job Offer",
    "Salary or Payroll Inquiry",
    "Benefits Inquiry",
    "Resignation & Exit",
    "Attendance & Timesheet",
    "Recruitment Process",
    "Policy Clarification",
    "Training & Development",
    "Work From Home Requests",
    "Relocation & Transfer",
    "Expense Reimbursement",
    "IT & Access Issues",
    "Events & Celebrations"
]

In [23]:
def fetch_unclassified_emails():
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """)
    return cursor.fetchall()

emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"
    result = classifier(email_text, categories)
    predicted_label = result["labels"][0]
    confidence = result["scores"][0]

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}")

    # Update the classified category in DB
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))

conn.commit()
print("✅ Reclassification complete with final 15 categories.")

📩 Email ID 1
🧠 Predicted: Leave Request
📈 Confidence: 0.71
📩 Email ID 2
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.31
📩 Email ID 3
🧠 Predicted: Salary or Payroll Inquiry
📈 Confidence: 0.32
📩 Email ID 7
🧠 Predicted: Leave Request
📈 Confidence: 0.74
📩 Email ID 10
🧠 Predicted: Onboarding
📈 Confidence: 0.26
📩 Email ID 11
🧠 Predicted: Leave Request
📈 Confidence: 0.74
📩 Email ID 8
🧠 Predicted: Leave Request
📈 Confidence: 0.74
📩 Email ID 4
🧠 Predicted: Leave Request
📈 Confidence: 0.71
📩 Email ID 5
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.31
📩 Email ID 6
🧠 Predicted: Salary or Payroll Inquiry
📈 Confidence: 0.32
📩 Email ID 12
🧠 Predicted: Leave Request
📈 Confidence: 0.74
✅ Reclassification complete with final 15 categories.


In [24]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli"
)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cpu


In [39]:
cursor.execute("""
    UPDATE emails
    SET classified_category = NULL
""")
conn.commit()

print("✅ All classified categories reset to NULL.")

✅ All classified categories reset to NULL.


In [26]:
def fetch_unclassified_emails():
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """)
    return cursor.fetchall()

emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"
    result = classifier(email_text, categories)
    predicted_label = result["labels"][0]
    confidence = result["scores"][0]

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}")

    # Update the classified category in DB
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))

conn.commit()
print("✅ Reclassification complete with final 15 categories.")

📩 Email ID 1
🧠 Predicted: Leave Request
📈 Confidence: 0.98
📩 Email ID 2
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.67
📩 Email ID 3
🧠 Predicted: Benefits Inquiry
📈 Confidence: 0.69
📩 Email ID 7
🧠 Predicted: Leave Request
📈 Confidence: 0.81
📩 Email ID 10
🧠 Predicted: Benefits Inquiry
📈 Confidence: 0.38
📩 Email ID 11
🧠 Predicted: Leave Request
📈 Confidence: 0.81
📩 Email ID 8
🧠 Predicted: Leave Request
📈 Confidence: 0.81
📩 Email ID 4
🧠 Predicted: Leave Request
📈 Confidence: 0.98
📩 Email ID 5
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.67
📩 Email ID 6
🧠 Predicted: Benefits Inquiry
📈 Confidence: 0.69
📩 Email ID 12
🧠 Predicted: Leave Request
📈 Confidence: 0.81
✅ Reclassification complete with final 15 categories.


## **Better performing model** 

In [35]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [38]:
# 📌 Label Map (Short → Descriptive)
label_map = {
    "Leave Request": "Requests related to taking leave or vacation",
    "Onboarding": "Questions about new hire onboarding or joining formalities",
    "Job Offer": "Inquiries regarding job offers or employment contracts",
    "Payroll Inquiry": "Questions related to salary, payslips, or payroll processing",
    "Benefits Inquiry": "Questions about insurance, medical, or employee benefits",
    "Resignation & Exit": "Emails about resignation, exit process, or final settlements",
    "Attendance & Timesheet": "Issues about work hours, attendance or timesheets",
    "Recruitment Process": "Questions about interview, screening or hiring stages",
    "Policy Clarification": "Clarification about company policies or procedures",
    "Training & Development": "Queries about training programs or skill development",
    "Work From Home Requests": "Requests or updates regarding remote work",
    "Relocation & Transfer": "Inquiries about internal transfers or relocation",
    "Expense Reimbursement": "Questions about reimbursements or expense claims",
    "IT & Access Issues": "Issues about system access, accounts, or technical problems",
    "Events & Celebrations": "Emails about office events, parties, or celebrations"
}

# 📌 Step 1: Extract Descriptive Prompts
descriptive_labels = list(label_map.values())

# 📌 Step 2: Fetch unclassified emails
def fetch_unclassified_emails():
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """)
    return cursor.fetchall()

# 📌 Step 3: Classify and update
emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"
    
    result = classifier(email_text, descriptive_labels)
    predicted_description = result["labels"][0]
    confidence = result["scores"][0]

    # Map back to clean label
    predicted_label = next(
        key for key, value in label_map.items() if value == predicted_description
    )

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}")

    # Update in DB
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))

conn.commit()
print("✅ All emails classified and updated using mapped descriptions.")

📩 Email ID 13
🧠 Predicted: Payroll Inquiry
📈 Confidence: 0.38
📩 Email ID 14
🧠 Predicted: Payroll Inquiry
📈 Confidence: 0.35
✅ All emails classified and updated using mapped descriptions.


In [ ]:
# 🧠 Mapped labels: actual category => descriptive version (for the classifier)
label_mapping = {
    "Leave Request": "An employee is requesting leave or time off.",
    "Onboarding": "A query related to joining process, documents, or orientation.",
    "Job Offer": "Discussion or questions about job offer details or status.",
    "Salary/Payroll Inquiry": "Inquiries related to salary, payroll, or payment delays.",
    "Benefits Inquiry": "Questions about employee benefits like insurance or perks.",
    "Resignation & Exit": "Emails related to resigning, exit process, or final settlement.",
    "Attendance & Timesheet": "Questions about logging attendance or timesheets.",
    "Recruitment Process": "Inquiries related to job applications or interviews.",
    "Policy Clarification": "Seeking clarification on company policies or HR rules.",
    "Training & Development": "Asking about learning opportunities or employee training programs.",
    "Work From Home Requests": "Request or questions related to working remotely or from home.",
    "Relocation & Transfer": "Discussion about relocation or internal transfer requests.",
    "Expense Reimbursement": "Questions about expense claims or reimbursement process.",
    "IT & Access Issues": "Problems accessing systems, tools, or login credentials.",
    "Events & Celebrations": "Emails about office events, celebrations, or social gatherings."
}

# 🧪 Descriptive labels list for classifier input
candidate_labels = list(label_mapping.values())


In [40]:
from transformers import pipeline

# 1. Descriptive label mapping
label_mapping = {
    "Leave Request": "An employee is requesting leave or time off.",
    "Onboarding": "A query related to joining process, documents, or orientation.",
    "Job Offer": "Discussion or questions about job offer details or status.",
    "Salary/Payroll Inquiry": "Inquiries related to salary, payroll, or payment delays.",
    "Benefits Inquiry": "Questions about employee benefits like insurance or perks.",
    "Resignation & Exit": "Emails related to resigning, exit process, or final settlement.",
    "Attendance & Timesheet": "Questions about logging attendance or timesheets.",
    "Recruitment Process": "Inquiries related to job applications or interviews.",
    "Policy Clarification": "Seeking clarification on company policies or HR rules.",
    "Training & Development": "Asking about learning opportunities or employee training programs.",
    "Work From Home Requests": "Request or questions related to working remotely or from home.",
    "Relocation & Transfer": "Discussion about relocation or internal transfer requests.",
    "Expense Reimbursement": "Questions about expense claims or reimbursement process.",
    "IT & Access Issues": "Problems accessing systems, tools, or login credentials.",
    "Events & Celebrations": "Emails about office events, celebrations, or social gatherings."
}
candidate_labels = list(label_mapping.values())

# 2. Initialize classifier
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=-1  # Use -1 for CPU, or 0 for GPU
)

# 3. Function to fetch unclassified emails
def fetch_unclassified_emails():
    query = """
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """
    cursor.execute(query)
    return cursor.fetchall()

# 4. Classify emails and update DB
emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    email_text = f"{subject}\n\n{body}"
    result = classifier(email_text, candidate_labels, multi_label=False)
    predicted_desc = result['labels'][0]
    confidence = result['scores'][0]

    # Reverse lookup
    predicted_label = next(
        key for key, value in label_mapping.items() if value == predicted_desc
    )

    # Update email record
    cursor.execute("""
        UPDATE emails
        SET classified_category = %s
        WHERE email_id = %s
    """, (predicted_label, email_id))
    conn.commit()

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}\n")

print("✅ Classification complete with descriptive labels.")

Device set to use cpu


📩 Email ID 11
🧠 Predicted: Leave Request
📈 Confidence: 0.83

📩 Email ID 8
🧠 Predicted: Leave Request
📈 Confidence: 0.83

📩 Email ID 4
🧠 Predicted: Leave Request
📈 Confidence: 0.84

📩 Email ID 5
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.27

📩 Email ID 1
🧠 Predicted: Leave Request
📈 Confidence: 0.84

📩 Email ID 2
🧠 Predicted: IT & Access Issues
📈 Confidence: 0.27

📩 Email ID 3
🧠 Predicted: Job Offer
📈 Confidence: 0.41

📩 Email ID 7
🧠 Predicted: Leave Request
📈 Confidence: 0.83

📩 Email ID 10
🧠 Predicted: Work From Home Requests
📈 Confidence: 0.2

📩 Email ID 6
🧠 Predicted: Job Offer
📈 Confidence: 0.41

📩 Email ID 12
🧠 Predicted: Leave Request
📈 Confidence: 0.83

📩 Email ID 13
🧠 Predicted: Policy Clarification
📈 Confidence: 0.33

📩 Email ID 14
🧠 Predicted: Salary/Payroll Inquiry
📈 Confidence: 0.4

✅ Classification complete with descriptive labels.
